In [60]:

import os
import copy
import pandas as pd

import numpy as np
def create_masked_data(dataDir, wriDir, tokenizer):
    '''
    create_masked_data('./MLM/mlm_prepared_data', './MLM/masked_data', tokenizer)
    '''
    def masking_sentence(token_ids, tokenizer):
        '''
        Function to mask each token in a sentence and return the masked sentence and the corresponding label
        '''
        except_tokens = [tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id]
        
        mask = np.random.binomial(1, 0.1, (len(token_ids),))   # Masking each token with 15% probability
        print("mask:",  np.where(mask)[0])
        token_ids_copy = copy.deepcopy(token_ids)
        masked_sentence = pd.DataFrame()
        labels = [-100] * len(token_ids)
        for idx, token in enumerate(token_ids):
            
            if (idx in np.where(mask)[0]) and (idx not in except_tokens):
                token_ids_copy[idx] = tokenizer.mask_token_id
                labels[idx] = token
            
        masked_sentence['masked_token_id'] = [token_ids_copy]
        masked_sentence['label_id'] = [labels]
            
        return masked_sentence
    
    train_file = os.path.join(dataDir, 'dev_mlm_abolish_full.json')
    train_data = pd.read_json(train_file, lines=True)
    
    masked_data_df = pd.DataFrame()
    for idx, sen in enumerate(train_data['token_id']):
        masked_sentence = masking_sentence(sen, tokenizer)
        
        masked_data_df = pd.concat([masked_data_df, masked_sentence], ignore_index=True)
    
    masked_data_df.to_json(os.path.join(wriDir, 'train_masked_data.json'), orient='records', lines=True)
    
    return masked_data_df

In [10]:
from transformers import AutoTokenizer
model_checkpoint = "dmis-lab/biobert-base-cased-v1.2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


0

In [61]:
create_masked_data('./MLM/mlm_prepared_data', './MLM/masked_data', tokenizer)

mask: [ 8 31]
mask: [11 31 34]
mask: [ 5 10 16 23 28 32 35]
mask: [ 3  8 29 45 46]
mask: [ 6 12 15 23]
mask: [ 9 15 18 27 28 42]
mask: [ 4  6 13 16 47]
mask: [15 18 21 26 30 38 46]
mask: [ 4 11 16 17 21 27 29 35 39 43 45]
mask: [35 44]
mask: [ 1  2  6 16 38 44 49]
mask: [ 9 16 18 38 43]
mask: [ 1  5  6 16 38]
mask: [11 29 34 45]
mask: [ 1  2 14 29 43 47]
mask: [27 29 36 38 49]
mask: [14 24 45]
mask: [ 2 12 18 19 20 23 25 26 35]
mask: [28 29 30 41 44]
mask: [ 3 11 21 41]


,masked_token_id,label_id
0,"[101, 1748, 4982, 117, 5190, 4789, 1107, 19255...","[-100, -100, -100, -100, -100, -100, -100, -10..."
1,"[101, 5871, 1105, 17331, 1182, 1707, 1118, 105...","[-100, -100, -100, -100, -100, -100, -100, -10..."
2,"[101, 1748, 4982, 117, 1103, 103, 26883, 1320,...","[-100, -100, -100, -100, -100, 3687, -100, -10..."
3,"[101, 27914, 1116, 103, 1144, 8632, 2393, 1643...","[-100, -100, -100, 1134, -100, -100, -100, -10..."
4,"[101, 1107, 5014, 117, 1748, 3687, 103, 1320, ...","[-100, -100, -100, -100, -100, -100, 26883, -1..."
5,"[101, 2393, 1643, 3246, 1110, 8632, 1118, 2791...","[-100, -100, -100, -100, -100, -100, -100, -10..."
6,"[101, 1103, 3687, 26883, 103, 1104, 103, 3031,...","[-100, -100, -100, -100, 1320, -100, 194, -100..."
7,"[101, 1103, 172, 1643, 1775, 10424, 1475, 1789...","[-100, -100, -100, -100, -100, -100, -100, -10..."
8,"[101, 181, 4559, 3080, 103, 1105, 2848, 14637,...","[-100, -100, -100, -100, 2007, -100, -100, -10..."
9,"[101, 17895, 1104, 1103, 5250, 23984, 176, 657...","[-100, -100, -100, -100, -100, -100, -100, -10..."


In [55]:
def masking_sentence(token_ids, tokenizer):
    '''
    Function to mask each token in a sentence and return the masked sentence and the corresponding label
    '''
    except_tokens = [tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id]
    
    mask = np.random.binomial(1, 0.1, (len(token_ids),))   # Masking each token with 15% probability
    print("mask:",  np.where(mask)[0])
    token_ids_copy = copy.deepcopy(token_ids)
    masked_sentence = pd.DataFrame()
    labels = [-100] * len(token_ids)
    for idx, token in enumerate(token_ids):
        
        if (idx in np.where(mask)[0]) and (idx not in except_tokens):
            token_ids_copy[idx] = tokenizer.mask_token_id
            labels[idx] = token
        
    masked_sentence['masked_token_id'] = [token_ids_copy]
    masked_sentence['label_id'] = [labels]
         
    return masked_sentence

In [62]:
token_id = [101,1107,5014,117,1748,3687,26883,1320,1104,1103,172,118,6020,14715,11179,16617,5777,1107,1103,185,7897,1571,21392,1116,171,1604,1105,171,1580,8632,15416,1348,23842,117,6142,4422,3687,26883,1320,1104,1103,21996,184,5822,11478,27105,17853,113,11769,102]

In [63]:
a = masking_sentence(token_id, tokenizer)

mask: [ 1  3 13 25 40]


In [58]:
a

,masked_token_id,label_id
0,"[101, 103, 4982, 117, 5190, 4789, 1107, 103, 1...","[-100, 1748, -100, -100, -100, -100, -100, 192..."


In [59]:
a['label_id'][0]

[-100,
 1748,
 -100,
 -100,
 -100,
 -100,
 -100,
 19255,
 -100,
 -100,
 -100,
 -100,
 1151,
 -100,
 -100,
 -100,
 1103,
 3687,
 26883,
 -100,
 -100,
 -100,
 -100,
 -100,
 117,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 1648,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100]

In [24]:
tokenizer.mask_token_id

103